In [1]:
import sys
sys.path.append("/Users/adi/School/Research/pipit")

import pipit.trace
from pipit.graph import Graph, Node

In [2]:
trace = pipit.trace.Trace.from_otf2("/Users/adi/School/Research/otf2_tracing/traces/Large_Score-P")

In [3]:
# can either add leave row index as calling context id or having a matching df indices column

graph = Graph()
callpath_to_node = dict()
graph_nodes = [None for i in range(len(trace.events))]
node_id = 0

for location_id in set(trace.events["Location ID"]):
    location_df = trace.events.loc[
        (trace.events["Name"] != "N/A")
        & (trace.events["Location ID"] == location_id)
    ]

    curr_depth = 0
    callpath = ""
    df_indices = list(location_df.index)
    function_names = list(location_df["Name"])
    event_types = list(location_df["Event Type"])
    functions_stack, nodes_stack  = [], []

    for i in range(len(location_df)):
        curr_df_index, evt_type, function_name = df_indices[i], event_types[i], function_names[i]

        if evt_type == "Enter":
            functions_stack.append(function_name)
            callpath = "->".join(functions_stack)

            if curr_depth == 0:
                parent_node = None
            else:
                parent_node = nodes_stack[-1]

            if callpath in callpath_to_node:
                curr_node = callpath_to_node[callpath]
            else:
                curr_node = Node(node_id, function_name, parent_node, curr_depth)
                callpath_to_node[callpath] = curr_node
                node_id += 1

                if curr_depth == 0:
                    graph.add_root(curr_node)
                else:
                    parent_node.add_child(curr_node)

            curr_node.add_calling_context_id(curr_df_index)
            graph.add_to_map(curr_df_index, curr_node)
            
            nodes_stack.append(curr_node)
            graph_nodes[curr_df_index] = curr_node
            curr_depth += 1
        else:
            curr_node = nodes_stack.pop()
            graph_nodes[curr_df_index] = curr_node

            functions_stack.pop()
            curr_depth -= 1

In [4]:
def print_helper(list_of_nodes, depth):
    for node in list_of_nodes:
        print(("---" * depth) + node.name)
        print_helper(node.children, depth + 1)

In [ ]:
"""
think about optimizing space by converting some of the attributes to None if dict is empty,
or string if dict only has one key/value, or strings, if only dicts of len 1 exist, etc
"""